### DOWNLOAD DE BIBLIOTECAS

In [ ]:
# FLASK
!pip install flask-ngrok

# RNA
!pip install Bio
!pip install numpy
!pip install pandas
!pip install pickle-mixin
!pip install tensorflow

### IMPORTAÇÕES

In [ ]:
# APP
from flask import Flask, make_response, render_template, request
from flask_ngrok import run_with_ngrok


# FUNÇÕES
from datetime import datetime, timedelta
import os, string, random, re

import smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import base64


# RNA
from Bio import SeqIO
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np

### PERMISSÃO PARA ACESSAR ARQUIVOS DO GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

google_drive = '/content/drive/MyDrive/Colab Notebooks/NCYPred/web/'

Mounted at /content/drive


### CLASSIFICADOR DE RNA

In [ ]:
# REMOVE SEQUÊNCIAS COM CARACTERES NÃO AUTORIZADOS
def remove(df):
    for i in range(len(df.seq)):
        for j in ['N', 'Y', 'K', 'W', 'R', 'H', 'M', 'S', 'D', 'V', 'B']:
            if j in df.seq[i]:
                df = df.drop(index=i)
                break

    df = df.reset_index(drop=True)
    return df


# DECOMPÕE A SEQUÊNCIA EM 3-MER
def seq_to_3mer(seq_list):
    main_list = []

    for _, i in enumerate(seq_list):
        seq = list(i)
        seq_kmer = []

        for j, _ in enumerate(seq):
            if j < len(seq) - 2:
                seq_kmer.append(seq[j] + seq[j+1] + seq[j+2])
            else:
                continue

        main_list.append(seq_kmer)

    return main_list


# TOKENIZAÇÃO
def token_pad(sentences, max_len, prefix):
    with open(google_drive + 'static/tokenize.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    tokens = tokenizer.texts_to_sequences(sentences)
    all_pad = pad_sequences(tokens, max_len, padding=prefix)

    return all_pad


# CONVERTE ARGMAX EM UM DICIONÁRIO
def argmax_to_label(predictions):
    label_list = ['5.8S-rRNA', '5S-rRNA', 'CD-box', 'HACA-box', 'Intron-gp-I', 'Intron-gp-II',
        'Leader', 'Riboswitch', 'Ribozyme', 'Y-RNA', 'Y-RNA-like', 'miRNA ', 'tRNA']

    argmax_pred = np.argmax(predictions, axis=1)
    argmax_values = range(13)
    pred_labels = []

    for i in argmax_pred:
        for j, k in zip(argmax_values, label_list):
            if i == j:
                pred_labels.append(k)

    return pred_labels, label_list


# MODELO DE PREDIÇÃO
def calculate(filename):
    # RENOMEIA O CAMINHO DOS ARQUIVOS
    input = google_drive + 'static/input/' + filename + '.fasta'
    output = google_drive + 'static/output/' + filename

    # DATAFRAME PARA ARMAZENAR OS DADOS
    df = pd.DataFrame()

    input_id, input_seq = [], []

    for seq_record in SeqIO.parse(input, 'fasta'):
        input_id.append(seq_record.id)

        if 'U' in seq_record.seq:
            dna_seq = seq_record.seq.back_transcribe()
            input_seq.append(dna_seq)

        else:
            input_seq.append(seq_record.seq)

    df['id'] = input_id
    df['seq'] = input_seq

    # REMOVE SEQUÊNCIAS COM CARACTERES NÃO PERMITIDOS
    df = remove(df)

    # DECOMPÕE A SEQUÊNCIA EM 3-MERS
    x = df['seq']
    x = seq_to_3mer(x)

    x_pad = token_pad(x, 498, 'post')

    # CARREGA O MODELO
    model = keras.models.load_model(google_drive + 'static/trained-model/', compile=False)
    predictions = model.predict(x_pad, verbose=0)
    pred_labels, label_list = argmax_to_label(predictions)

    df['prediction'] = pred_labels
    df_softmax = pd.DataFrame(data=predictions, columns=label_list)
    df_final = pd.concat([df, df_softmax], axis=1)

    # SALVA E RETORNA A SAÍDA
    df_final.to_csv('{}.csv'.format(output), sep=';', float_format='%.3f')
    return output + '.csv'

### FUNÇÕES

In [ ]:
# ATIVAR O ENVIO DE E-MAILS DO GOOGLE PELO SITE: https://www.google.com/settings/security/lesssecureapps
email_address = base64.b64decode(b'bmN5cHJlZEB1ZW0uYnI=').decode('ascii')
email_password = base64.b64decode(b'NDE4Y2I4MDQ=').decode('ascii')


# VERIFICA SE O E-MAIL É VÁLIDO
def check_email(email):
    regex = '^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$'
    return re.search(regex, email)


# ENVIA UMA MENSAGEM PARA O E-MAIL
def send_email(receiver, path):
    message = MIMEMultipart()
    message['Subject'] = 'NCYPred - Result of prediction'
    message['From'] = email_address
    message['To'] = receiver

    text = '''
Hello, the prediction results are ready, just download the file attached to this email.

Thank you very much for using our platform!

NCYPred Team.''' # VERSÃO EM TEXTO SIMPLES

    html = '''
<p>Hello, the prediction results are ready, just download the file attached to this email.</p>
<p>Thank you very much for using our platform!</p>
<p>NCYPred Team.</p>''' # VERSÃO EM TEXTO HTML

    # TRANSFORMA O TEXTO E O HTML EM MIMETEXT SIMPLES
    part = MIMEText(text, 'plain') # part = MIMEText(html, 'html')

    # ADICIONA O TEXTO E O HTML À MENSAGEM MIMEMULTIPART
    message.attach(part)

    # ABRE O ARQUIVO NO MODO BINÁRIO E CODIFICA O ANEXO
    file = open(path, 'rb')
    payload = MIMEBase('application', 'octate-stream', Name=os.path.basename(path))
    payload.set_payload((file).read())
    encoders.encode_base64(payload)

    # ADICIONA AO CABEÇALHO O ARQUIVO
    payload.add_header('Content-Decomposition', 'attachment', filename=os.path.basename(path))
    message.attach(payload)

    # CRIA UMA CONEXÃO SEGURA COM O SERVIDOR E ENVIA O E-MAIL
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as server: # with smtplib.SMTP('smtp.gmail.com: 587') as server:
        try:
            server.ehlo() # server.starttls()
            server.login(email_address, email_password)
            server.sendmail(email_address, receiver, message.as_string())
            server.quit()
            return True
        except:
            return False


# SALVA O ARQUIVO NA PASTA ESTÁTICA DE ENTRADA
def save_file(file):
    name = datetime.now().strftime('%Y%m%d%H%M%S') + ''.join(random.choice(string.ascii_letters) for i in range(6)) + file.filename
    path = os.path.join(google_drive, 'static', 'input', name)

    file.save(path)
    if os.path.isfile(path):
        return os.path.splitext(name)[0]
    return ''


# SALVA O TEXTO EM UM ARQUIVO NA PASTA ESTÁTICA DE ENTRADA
def create_file(text):
    name = datetime.now().strftime('%Y%m%d%H%M%S') + ''.join(random.choice(string.ascii_letters) for i in range(6)) + '.fasta'
    path = os.path.join(google_drive, 'static', 'input', name)

    file = open(path, 'w')
    rows = file.write(text)
    file.close()

    if rows:
        return os.path.splitext(name)[0]
    return ''


# REMOVE OS ARQUIVOS QUE ESTÃO HÁ MUITOS DIAS NO SERVIDOR
def clean_files(days):
    root = os.path.abspath('.')
    input_path = os.path.join(google_drive, 'static', 'input', '')
    output_path = os.path.join(google_drive, 'static', 'output', '')

    input_dir = os.listdir(input_path)
    output_dir = os.listdir(output_path)

    current_date = datetime.now() - timedelta(days=days)

    for i in input_dir:
        day = datetime(int(i[:4]), int(i[4:6]), int(i[6:8]))
        if day <= current_date:
            os.remove(input_path + i)

    for o in output_dir:
        day = datetime(int(o[:4]), int(o[4:6]), int(o[6:8]))
        if day <= current_date:
            os.remove(output_path + o)

### APP

In [ ]:
app = Flask(__name__, template_folder=google_drive + 'templates', static_folder=google_drive + 'static')
run_with_ngrok(app)
project = 'NCYPred'


# VIEW - PÁGINA SOBRE O PROJETO
@app.route('/about')
def about():
    page, title = 'about', 'About'
    return render_template('about.html', page=page, project=project, title=title)


# VIEW - PÁGINA DE ENVIO (PÁGINA INICIAL)
# ACTION - PROCESSA O ARQUIVO OU TEXTO FORNECIDO E RETORNA UM JSON COM OS RESULTADOS
@app.route('/')
@app.route('/index')
@app.route('/submit', methods=['GET', 'POST'])
def submit():
    page, title = 'submit', 'Submit'

    if request.method == 'GET':
        return render_template('submit.html', page=page, project=project, title=title)

    elif request.method == 'POST':
        data = function()
        return render_template('submit.html', page=page, project=project, title=title, message=data['message'], status=data['status'], data=data['data'])


# VIEW - PÁGINA DE AJUDA
@app.route('/help')
def help():
    page, title = 'help', 'Help'
    return render_template('help.html', page=page, project=project, title=title)


# VIEW - PÁGINA DE EXPLICAÇÃO DO ALGORITMO
@app.route('/algorithm')
def algorithm():
    page, title = 'algorithm', 'Algorithm'
    return render_template('algorithm.html', page=page, project=project, title=title)


# VIEW - PÁGINA DO TIME
@app.route('/team')
def team():
    page, title = 'team', 'Team'
    return render_template('team.html', page=page, project=project, title=title)


# VIEW - PÁGINA DE CONTATO
@app.route('/contact')
def contact():
    page, title = 'contact', 'Contact'
    return render_template('contact.html', page=page, project=project, title=title)

# VIEW - PÁGINA DE ERRO
@app.errorhandler(404)
def error(error):
	return submit()


# ACTION - PROCESSA O ARQUIVO OU TEXTO FORNECIDO E RETORNA UM JSON COM OS RESULTADOS
@app.route('/process', methods=['POST'])
def process():
	if request.method == 'POST':
		data = function()

		response = make_response(data)
		response.headers['Content-Type'] = 'application/json'
		return response

	else:
		return ''


# FUNCTION - PROCESSA O ARQUIVO OU TEXTO FORNECIDO
def function():
    data = {'data': ''}

    if 'file' in request.files and request.files['file'].filename != '': # ENVIOU UM ARQUIVO (IGNORA CASO TENHA ENVIADO UMA SEQUÊNCIA DE TEXTO)
        file = request.files['file']
        path = save_file(file)
        status = bool(path)

        try: # TENTA CALCULAR A SEQUÊNCIA INFORMADA
            path = calculate(path)
            data['message'] = 'The provided file has been processed. ' # O ARQUIVO FORNECIDO FOI PROCESSADO
            status = True

        except: # A SEQUÊNCIA DE RNA INFORMADA POSSUI ALGUM ERRO
            data['message'] = 'Unable to process, there is an error in the file provided.' # NÃO FOI POSSÍVEL PROCESSAR, HÁ UM ERRO NO ARQUIVO FORNECIDO
            status = False

    elif 'sequence' in request.form and len(request.form['sequence']) > 0: # ENVIOU UMA SEQUÊNCIA DE TEXTO
        sequence = request.form['sequence'].strip()

        try: # TENTA CALCULAR A SEQUÊNCIA INFORMADA
            input = create_file(sequence)
            path = calculate(input)
            data['message'] = 'The reported string has been processed. ' # A SEQUÊNCIA INFORMADA FOI PROCESSADA
            status = True

        except: # A SEQUÊNCIA DE RNA INFORMADA POSSUI ALGUM ERRO
            data['message'] = 'Could not process, there is an error in the reported string.' # NÃO FOI POSSÍVEL PROCESSAR, HÁ UM ERRO NA SEQUÊNCIA INFORMADA
            status = False

    else: # NÃO ENVIOU UMA SEQUÊNCIA DE TEXTO
        data['message'] = 'Unable to process, no RNA sequences were reported.' # NÃO FOI POSSÍVEL PROCESSAR, NENHUMA SEQUÊNCIA DE RNA FOI INFORMADA
        status = False

    if status and 'email' in request.form and len(request.form['email']) > 0: # INFORMOU UM E-MAIL
        email = request.form['email'].strip()

        if check_email(email): # O E-MAIL INFORMADO É VÁLIDO
            if send_email(email, path): # O SERVIDOR CONSEGUIU ENVIAR O E-MAIL
                data['message'] += 'An email with the results has been sent to your message box.' # UM E-MAIL COM OS RESULTADOS FOI ENVIADO PARA SUA CAIXA DE MENSAGENS

            else: # HOUVE UM PROBLEMA NO SERVIDOR AO ENVIAR O E-MAIL
                data['message'] += 'It was not possible to send the email, please download the result file.' # NÃO FOI POSSÍVEL ENVIAR O E-MAIL, FAÇA O DOWNLOAD DO ARQUIVO DE RESULTADO
                data['data'] = '<a href="/static/output/' + os.path.basename(path) + '">Click here to download.</a>' # CLIQUE AQUI PARA FAZER O DOWNLOAD

        else: # O E-MAIL INFORMADO É INVÁLIDO
            data['message'] += 'The email provided is invalid, please download the result file.'
            data['data'] = '<a href="/static/output/' + os.path.basename(path) + '">Click here to download.</a>' # CLIQUE AQUI PARA FAZER O DOWNLOAD

    elif status: # NÃO INFORMOU UM E-MAIL
        data['message'] += 'A link that will last 3 days was generated.'
        data['data'] = '<a href="/static/output/' + os.path.basename(path) + '">Click here to download.</a>' # CLIQUE AQUI PARA FAZER O DOWNLOAD

    data['status'] = status
    return data


# VIEW - PÁGINA DE LIMPEZA DE ARQUIVOS ANTIGOS
@app.route('/clean')
def clean():
    clean_files(3)
    return ''

### RUN

In [ ]:
app.run()